### UAS - Text Mining
#### Nama: Aurellia Vieryn <br>
#### NIM: 2440066626
#### Video Link: https://binusianorg-my.sharepoint.com/personal/aurellia_vieryn001_binus_ac_id/_layouts/15/guestaccess.aspx?docid=055768b8d36a744ba88e5021b6f2de770&authkey=AdCFe6gWs9nKQJjpleRbFv8&e=mqmUQs

Pada soal UAS untuk mata kuliah Text Mining, diberikan 4 buah dataset sesuai dengan NIM dan gender. Saya perempuan dan memiliki NIM genap, maka dataset yang akan digunakan adalah dataset D. <br>

Untuk soal yang diberikan kali ini ada 4 soal kasus yang meliputi Text Classification, Topic Modelling, dan Text Summarization.

#### No 1 - Classification Model

##### Loading and Preprocessing

In [1]:
import pandas as pd
data1 = pd.read_csv('data_1D.csv')

In [2]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12606 entries, 0 to 12605
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12606 non-null  int64 
 1   text        12606 non-null  object
 2   label       12606 non-null  object
dtypes: int64(1), object(2)
memory usage: 295.6+ KB


In [3]:
data1.label.unique()

array(['Electronics', 'Clothing & Accessories', 'Books', 'Household'],
      dtype=object)

In [4]:
data1.text

0        Tamron AF 70-300mm F/4.0-5.6 Di LD Macro Telep...
1               Softline Butterfly Women's Cotton Camisole
2        Indian Economy: For UPSC Civil Services & Othe...
3        Fujifilm X-T3 Mirrorless Digital Camera with X...
4        Barata Formal Ties For Men, Navy Blue Tie Form...
                               ...                        
12601    EK UDAAN Unisex Woolen Thermal Wear Thumb Sock...
12602    Prime-Comfort (Blue and Yellow) Premium Foldab...
12603    The One-Straw Revolution: An Introduction to N...
12604    Peter Jones Full Rim Round Unisex Spectacle Fr...
12605    Crompton Bliss 3-Litre Instant Water Heater (W...
Name: text, Length: 12606, dtype: object

Hal pertama yang akan dilakukan setelah mengimport dataset adalah melihat dataset yang akan digunakan. Dataset ini adalah dataset tentang deskripsi produk pada sebuah website e-commerce pada `text` column, dan label yang dimiliki oleh `target` column ada 4, yaitu Household, Books, Electronics dan Clothing & Accessories. <br>

In [5]:
import re
import string

def data_cleansing(df):
    clean_data = df
    # Make Lowercase
    clean_data.text = df.text.str.lower()
    # Remove \n in Text
    clean_data.text = [re.sub(r'\n', ' ', i) for i in clean_data.text]
    # Punctuations
    clean_data.text = [word.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) for word in clean_data.text]
    # Remove Numbers
    clean_data.text = [re.sub(r"\d+","", i) for i in clean_data.text]
    # Remove Single Characters
    clean_data.text = [re.sub(r'(?:^| )\w(?:$| )', ' ', i).strip() for i in clean_data.text]
    # Remove Multiple Whitespaces
    clean_data.text = [re.sub(r"\s+"," ", i) for i in clean_data.text]

    return clean_data

In [6]:
data1 = data_cleansing(data1)

In [7]:
data1.text

0        tamron af mm di ld macro telephoto zoom lens f...
1                 softline butterfly women cotton camisole
2        indian economy for upsc civil services other s...
3        fujifilm t mirrorless digital camera with xf m...
4        barata formal ties for men navy blue tie forma...
                               ...                        
12601    ek udaan unisex woolen thermal wear thumb sock...
12602    prime comfort blue and yellow premium foldable...
12603    the one straw revolution an introduction to na...
12604    peter jones full rim round unisex spectacle fr...
12605    crompton bliss litre instant water heater whit...
Name: text, Length: 12606, dtype: object

Setelah itu, data `text` pada dataset akan dilakukan proses preprocessing yang meliputi lowercase dari text, menghilangkan newline, tanda baca, angka, karakter tunggal, dan whitespace yang lebih dari 1.

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
data1.text = data1.apply(lambda row: nltk.word_tokenize(row.text), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')
list_stopwords = set(stopwords.words('english'))
data1.text = data1.text.apply(lambda x: [item for item in x if item not in list_stopwords])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
data1.text

0        [tamron, af, mm, di, ld, macro, telephoto, zoo...
1           [softline, butterfly, women, cotton, camisole]
2        [indian, economy, upsc, civil, services, state...
3        [fujifilm, mirrorless, digital, camera, xf, mm...
4        [barata, formal, ties, men, navy, blue, tie, f...
                               ...                        
12601    [ek, udaan, unisex, woolen, thermal, wear, thu...
12602    [prime, comfort, blue, yellow, premium, foldab...
12603    [one, straw, revolution, introduction, natural...
12604    [peter, jones, full, rim, round, unisex, spect...
12605    [crompton, bliss, litre, instant, water, heate...
Name: text, Length: 12606, dtype: object

Kolom `text` lalu akan diubah set token dan akan dihilangkan token-token yang merupakan stopword (kata-kata yang dianggap tidak penting). Dapat dilihat dari hasilnya kalau kolom `text` sudah berbentuk token.

##### Count Vectorization and Naive Bayes

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data1.text, data1.label, test_size=0.2, random_state=42)

Dataset yang telah dibersihkan akan dibagi menjadi training and testing untuk pengujian model. Metode vektorisasi yang akan digunakan untuk section ini adalah Count Vectorizer (dihitung dari frekuensi kemunculan setiap kata dalam teks) dengan Model Naive Bayes (teorema Bayes dengan asumsi naive).

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform([' '.join(tokens) for tokens in x_train])
x_test = vectorizer.transform([' '.join(tokens) for tokens in x_test])

In [13]:
x_train.shape

(10084, 37601)

In [14]:
y_train.shape

(10084,)

In [15]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(x_train, y_train)

MultinomialNB()

In [16]:
y_pred = naive_bayes.predict(x_test)

In [17]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9480570975416336


Dengan kombinasi CountVectorizer dan model Naive Bayes, hasil yang didapatkan adalah akurasi sebesar 94.8%. Hasil ini tergolong baik untuk akurasi sebuah model.

##### TF IDF Vectorizer and SVM Model

In [18]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(data1.text, data1.label, test_size=0.2, random_state=42)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer2 = TfidfVectorizer()
x_train2 = vectorizer2.fit_transform([' '.join(tokens) for tokens in x_train2])
x_test2 = vectorizer2.transform([' '.join(tokens) for tokens in x_test2])

Metode vektorisasi yang akan digunakan untuk section selanjutnya adalah TF-IDF Vectorizer (dihitung dari frekuensi kemunculan setiap kata dalam teks dan juga pentingnya sebuah kata) dengan SVM (Support Vector Machine).

In [20]:
from sklearn.svm import SVC

svm_model = SVC()
svm_model.fit(x_train2, y_train2)

SVC()

In [21]:
y_pred2 = svm_model.predict(x_test2)

In [22]:
accuracy = accuracy_score(y_test2, y_pred2)
print("Accuracy:", accuracy)

Accuracy: 0.9623314829500397


Dengan kombinasi TfIdfVectorizer dan model SVM, hasil yang didapatkan adalah akurasi sebesar 96.23%. Hasil ini lebih baik daripada percobaan yang dilakukan sebelumnya.

##### Conclusion
Hasil akurasi model klafisikasi CountVectorizer - Naive Bayes adalah 94.8% dan hasil model klasifikasi TfIdfVectorizer adalah 96.23%. Ini artinya model 1 berhasil mengklasifikasi test data produk sebesar 94.8% dan model 2 berhasil mengklasifikasi test data produk sebesar 96.23%. Model yang lebih baik adalah model kedua karena memiliki hasil akurasi yang lebih tinggi.

#### No 2 - Model LLM: BERT

In [ ]:
!pip install transformers

In [ ]:
! pip install optuna

Untuk model LLM yang akan dipakai di kasus ini adalah BERT (Bidirectional Encoder Representations from Transformers). Model ini dipilih karena BERT adalah salah satu model LLM yang populer, dan juga model ini sudah dilakukan pretraining pada jumlah teks yang besar terlebih dahulu. Setelah itu, BERT dilakukan fine tuning untuk tugas tertentu, dimana tugas itu di sini adalah text classification. <br>

BERT sendiri juga memiliki banyak keunggulan, seperti BERT yang bisa memahami konteks dan hubungan antar kata pada teks, dapat menghasilkan representasi teks yang baik, dan juga bisa mengetahui makna dari bahasanya.

In [25]:
import tensorflow as tf
import transformers
import torch
import transformers
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# Convert tokens to input tensors
def preprocess_tokens(tokens):
    input_ids = []
    attention_mask = []
    for text in tokens:
        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoded_text['input_ids'][0])
        attention_mask.append(encoded_text['attention_mask'][0])
    input_ids = tf.stack(input_ids)
    attention_mask = tf.stack(attention_mask)
    return input_ids, attention_mask

# Split the data into train and test sets
X_train_tokens, X_test_tokens, y_train, y_test = train_test_split(data1.text, data1.label, test_size=0.2, random_state=42)

# Preprocess the token data
X_train_input_ids, X_train_attention_mask = preprocess_tokens(X_train_tokens)
X_test_input_ids, X_test_attention_mask = preprocess_tokens(X_test_tokens)

# Apply label encoding to the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to one-hot encoded vectors
y_train_processed = tf.keras.utils.to_categorical(y_train_encoded, num_classes=4)
y_test_processed = tf.keras.utils.to_categorical(y_test_encoded, num_classes=4)

def objective(trial):
    # Define the hyperparameters to be tuned
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    num_train_epochs = trial.suggest_int('num_train_epochs', 4, 5)

    model = transformers.TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
    input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32)
    output = model([input_ids, attention_mask])
    output = output.logits
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # Train the model
    model.fit([X_train_input_ids, X_train_attention_mask], y_train_processed, batch_size=16, epochs=num_train_epochs)

    # Evaluate the model on the test set
    _, accuracy = model.evaluate([X_test_input_ids, X_test_attention_mask], y_test_processed)
    print("Test Accuracy:", accuracy)

    return accuracy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Perform hyperparameter optimization using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

# Get the best hyperparameters and accuracy
best_params = study.best_params
best_accuracy = study.best_value

print("Best hyperparameters:", best_params)
print("Best accuracy:", best_accuracy)

[I 2023-07-08 19:26:48,992] A new study created in memory with name: no-name-6aaa13f1-5da6-4c2b-86be-6292d9a91779
<ipython-input-25-c95d6e7a9ce4>:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4
631/631 [==============================] - 348s 468ms/step - loss: 0.3031 - accuracy: 0.9160
Epoch 2/4
631/631 [==============================] - 277s 440ms/step - loss: 0.2016 - accuracy: 0.9505
Epoch 3/4
631/631 [==============================] - 274s 434ms/step - loss: 0.1452 - accuracy: 0.9657
Epoch 4/4
79/79 [==============================] - 27s 298ms/step - loss: 0.2039 - accuracy: 0.9524


[I 2023-07-08 19:47:02,710] Trial 0 finished with value: 0.9524187445640564 and parameters: {'learning_rate': 9.085267291205254e-05, 'num_train_epochs': 4}. Best is trial 0 with value: 0.9524187445640564.


Test Accuracy: 0.9524187445640564


...

Test Accuracy: 0.964710533618927


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4
631/631 [==============================] - 329s 443ms/step - loss: 0.3068 - accuracy: 0.9135
Epoch 2/4
631/631 [==============================] - 274s 434ms/step - loss: 0.1419 - accuracy: 0.9631
Epoch 3/4
631/631 [==============================] - 273s 433ms/step - loss: 0.0869 - accuracy: 0.9774
Epoch 4/4
79/79 [==============================] - 29s 307ms/step - loss: 0.1693 - accuracy: 0.9627


[I 2023-07-08 21:15:22,648] Trial 4 finished with value: 0.962727963924408 and parameters: {'learning_rate': 1.467456019125218e-05, 'num_train_epochs': 4}. Best is trial 3 with value: 0.964710533618927.


Test Accuracy: 0.962727963924408
Best hyperparameters: {'learning_rate': 6.497931429224871e-05, 'num_train_epochs': 4}
Best accuracy: 0.964710533618927


Kode di atas adalah proses hyperparameter tuning dari model BERT menggunakan optuna. Prosesnya meliputi:
- Pengambilan model pretraining BERT base uncased
- Splitting ke training dan testing data
- Preprocessing kolom `text` agar bisa diproses untuk model BERT, yaitu membuat `input_id` dan `attention_mask`
- Pengubahan kolom `label` menjadi one-hot encoded vector dengan `LabelEncoder` dan `to_categorical`
-  Pembuatan function `objective` optuna dengan mendefine hyperparameter yang mau ditune, modelnya, dan hasil evaluasi dari training modelnya. Hyperparameter yang dituning adalah learning rate dan epoch karena berpengaruh pada modelnya, dimana learning rate adalah besar perubahan yang dilakukan pada weights modek, dan epoch adalah jumlah iterasi dari training set. Pada proses ini juga akan dilakukan proses finetuning yang menyesuaikan model untuk tugas text classification.
- Pembuatan study optuna dengan memilih yang paling besar (`maximize`), dan optimisasi studynya menggunakan function `objective` serta jumlah trialnya 5 saja agar prosesnya tidak memakan waktu yang banyak
- Hasil dari parameter paling baik serta akurasinya
<br>

##### Conclusion
Dari hasilnya yang diiterasi sebanyak 5 kali antar hyperparameter, didapatkan hyperparameter yang terbaik adalah learning rate 6.5e-05 dan jumlah epochsnya 4. Akurasi dari model BERT dengan hyperparameter tersebut adalah sekitar 96.471%. Artinya untuk testing dataset berhasil terklasfikasi secara akurat sebanyak 96.471%.


#### No 3 - Topic Modelling

In [4]:
import pandas as pd
data = pd.read_csv('data_3D.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   ABSTRACT    100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [5]:
data.head()

,Unnamed: 0,ABSTRACT
0,300,One of the popular approaches for low-rank t...
1,301,Nous tentons dans cet article de proposer un...
2,302,X-ray computed tomography (CT) using sparse ...
3,303,A singular (or Hermann) foliation on a smoot...
4,304,The Weyl semimetal phase is a recently disco...


Dapat dilihat dari informasi datasetnya bahwa tidak ada label pada dataset, hanya ada sebuah kolom teks dan mungkin ID. Karena ID tersebut tidak penting, maka akan dibuat dataset baru yang hanya berisi kolom teksnya.

In [6]:
data2 = data.ABSTRACT
import re
import string

def data_cleansing(df):
    clean_data = df
    # Make Lowercase
    clean_data = df.str.lower()
    # Remove \n in Text
    clean_data = [re.sub(r'\n', ' ', i) for i in clean_data]
    # Punctuations
    clean_data = [word.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) for word in clean_data]
    # Remove Numbers
    clean_data = [re.sub(r"\d+","", i) for i in clean_data]
    # Remove Single Characters
    clean_data = [re.sub(r'(?:^| )\w(?:$| )', ' ', i).strip() for i in clean_data]
    # Remove Multiple Whitespaces
    clean_data = [re.sub(r"\s+"," ", i) for i in clean_data]

    return clean_data

data3 = data_cleansing(data2)

Seperti pada soal-soal sebelumnya, data dipreproses terlebih dahulu. Langkah preprocessing ini sama pada langkah preprocessing yang dilakukan untuk nomor pertama.

In [7]:
import gensim
from gensim import corpora

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [8]:
data_processed=[]
for i in range(len(data3)):
    data_processed.append(preprocess(data3[i]))

Proses tokenisasi pada kasus ini akan menggunakan predefined function dari gensim yang juga otomatis menghilangkan stopwords yang ada di dalamnya.

In [9]:
from gensim.corpora import Dictionary
def build_dic(text):
    dictionary = Dictionary(text)
    dictionary.filter_extremes(no_below=1, no_above=0.2, keep_n= 100000)
    return dictionary

In [10]:
dictionary = build_dic(data_processed)
corpus = [dictionary.doc2bow(doc) for doc in data_processed]

Setelah itu, kita akan membuat Dictionary yang didasarkan dari teks yang sudah bersih, dan juga membuat korpus dari dictionarynya dengan Bag-of-Words representation.

In [11]:
from gensim import models

def vector_tfidf(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    return corpus_tfidf

corpus_tfidf = vector_tfidf(corpus)

Corpus yang tadi akan dikonversi lagi menggunakan metode vektorisasi TF-IDF untuk meningkatkan kualitas seperti memudahkan interpretasi, mengurangi dimensionality, mengurangi noise, dan juga meningkatkan fleksibilitas.

In [12]:
from gensim.models import CoherenceModel

num_topics_list = [2,3,4,5,6,7,8,9,10]

coherence_scores = []

for num_topics in num_topics_list:
    lda_model = gensim.models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=10, alpha=0.7, eta=0.7, random_state=42)
    coherence_model = CoherenceModel(model=lda_model, texts=data_processed, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    coherence_scores.append(coherence_score)

best_index = coherence_scores.index(max(coherence_scores))
best_num_topics = num_topics_list[best_index]

print("Best number of topics:", best_num_topics)

Best number of topics: 2


Analisa jumlah topik akan dilakukan menggunakan perbandingan coherence skor, dimana semakin tinggi nilai coherence maka akan semakin bagus karena topik akan semakin bermakna. Model yang digunakan di sini adalah model unsupervised LDA (Latent Dirichlet Allocation) yang biasa digunakan untuk kasus topic modelling yang mencari topik dari sejumlah dokumen. <br>

Pada kodenya, akan dibandingan jumlah topik dari 2 topik hingga 10 topik. Hasilnya menunjukkan bahwa jumlah topik yang paling baik untuk dataset ini adalah 2 topik saja.

In [13]:
num_topics = 2
lda_model = gensim.models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=10, alpha=0.7, eta=0.7, random_state=42)

from gensim.models import CoherenceModel
coherence_model = CoherenceModel(model=lda_model, texts=data_processed, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print("Coherence Score:", coherence_score) # Closer to 1, the better

Coherence Score: 0.4802055790467368


Bisa dilihat bahwa dengan 2 topik, coherence skor yang didapat adalah 0.48. Ini artinya kata-kata pada setiap topik berkorelasi sekitar 48%. Hasil ini masih bisa ditingkan tapi karena ini adalah hasil tertinggi yang didapatkan, maka kita tentukan 2 topik untuk dataset tersebut.

In [14]:
top_words_per_topic = lda_model.print_topics(num_words=10)
top_words_per_topic

[(0,
  '0.001*"learning" + 0.001*"network" + 0.001*"deep" + 0.001*"networks" + 0.001*"quantum" + 0.001*"neural" + 0.001*"models" + 0.001*"order" + 0.001*"state" + 0.001*"fusion"'),
 (1,
  '0.001*"delta" + 0.001*"bound" + 0.001*"operator" + 0.001*"algorithm" + 0.001*"graded" + 0.001*"measures" + 0.001*"type" + 0.001*"distance" + 0.001*"mathbf" + 0.001*"function"')]

Kita juga dapat lihat kata-kata yang memiliki peranan terpenting pada setiap topik dan jumlahnya sama semua. Mungkin ini artinya tidak ada kata yang memiliki peran yang sangat penting. Model LDA yang tadi akan kita buat menjadi file `pkl`.

In [15]:
import pickle as pkl
filename = 'lda_model.pkl'
pkl.dump(lda_model, open(filename, 'wb'))

In [16]:
X_test = data2.iloc[10:15]
clean_data = data_cleansing(X_test)

Dengan model yang tadi, akan dicoba untuk mengenerate topic pada contoh training data (diambil dari row 10-15) menjadi testing data. Data akan dilakukan tahap-tahap yang sama dengan sebelumnya.

In [17]:
process = []
for i in range(len(clean_data)):
    process.append(preprocess(clean_data[i]))

corpus_test = [dictionary.doc2bow(doc) for doc in process]
corpus_tfidfTest = vector_tfidf(corpus_test)

In [18]:
lda_model= pkl.load(open(filename, 'rb'))

In [19]:
test_shape=len(clean_data)
topic1=[]
score1=[]
topic2=[]
score2=[]
sentence=[]

for doc in range(test_shape):
    sentence.append(clean_data[doc])
    topic1.append(sorted(lda_model[corpus_test[doc]], key=lambda tup: -1*tup[1])[0][0])
    score1.append(sorted(lda_model[corpus_test[doc]], key=lambda tup: -1*tup[1])[0][1])
    topic2.append(sorted(lda_model[corpus_test[doc]], key=lambda tup: -1*tup[1])[1][0])
    score2.append(sorted(lda_model[corpus_test[doc]], key=lambda tup: -1*tup[1])[1][1])

sentence=pd.DataFrame(sentence,columns=['sentence']).reset_index()
topic1=pd.DataFrame(topic1,columns=['topic1']).reset_index()
score1=pd.DataFrame(score1,columns=['score1']).reset_index()
topic2=pd.DataFrame(topic2,columns=['topic2']).reset_index()
score2=pd.DataFrame(score2,columns=['score2']).reset_index()
test_result=pd.concat([sentence,topic1,score1,topic2,score2],axis=1)

test_result=test_result[['sentence','topic1','score1','topic2','score2']]
test_result.head()

,sentence,topic1,score1,topic2,score2
0,we report the discovery of three small transit...,1,0.981540,0,0.018461
1,we define family of quantum invariants of clos...,0,0.983426,1,0.016571
2,we propose sparse neural network architectures...,0,0.987163,1,0.012837
3,computer vision has made remarkable progress i...,0,0.981278,1,0.018723
4,numerical analysis of heat conduction through ...,0,0.984365,1,0.015628


Dari hasil testing model di atas, dapat dilihat skor probabilitas sebuah kalimat masuk pada topik terentu dan juga topik yang pada akhirnya ditentukan untuk kalimat tersebut. Hasil di atas menunjukkan probabilitas yang sangat besar sekitar 98% untuk setiap kalimat pada skor untuk topik 1, namun yang tergolong pada topik 1 hanyalah kalimat yang pertama.

In [20]:
persona_topics = lda_model[corpus]

# Assign personas based on dominant topic
personas = {}
for i, doc_topics in enumerate(persona_topics):
    dominant_topic = max(doc_topics, key=lambda x: x[1])[0]
    if dominant_topic not in personas:
        personas[dominant_topic] = []
    personas[dominant_topic].append(data3[i])

# Print the personas
for topic, persona in personas.items():
    print(f"Persona {topic+1}:")
    for text in persona:
        print(text)
    print('\n')

Persona 1:
one of the popular approaches for low rank tensor completion is to use the latent trace norm regularization however most existing works in this direction learn sparse combination of tensors in this work we fill this gap by proposing variant of the latent trace norm that helps in learning non sparse combination of tensors we develop dual framework for solving the low rank tensor completion problem we first show novel characterization of the dual solution space with an interesting factorization of the optimal solution overall the optimal solution is shown to lie on cartesian product of riemannian manifolds furthermore we exploit the versatile riemannian optimization framework for proposing computationally efficient trust region algorithm the experiments illustrate the efficacy of the proposed algorithm on several real world datasets across applications
nous tentons dans cet article de proposer une thèse cohérente concernant la formation de la notion involution dans le brouillo

Untuk mengakhiri kasus topik modelling ini, kita akan melakukan persona analysis. Persona pada dataset di atas adalah 2 karena topiknya ada 2. Persona analisis dilakukan dengan memeriksa topik-topik yang dominan dan hasil ini dapat digunakan untuk mengetahui lebih banyak tentang karakteristik mereka.

#### No 4 - Text Summarization

In [39]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [40]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    #split the sentences based on dot (.) separator
    article = filedata[0].split(". ")
    sentences = []
    for sentence in article:
        print(sentence)
        #remove non alphabetic character
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()

    return sentences

Function di atas adalah function untuk memproses file `.txt` dengan memisahkan teks menjadi kalimat dan juga langung melakukan tahap preprocessing yang simpel.

In [41]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    #build vector 0 with dimension following the len of all_words
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

In [42]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

Function `sentence_similarity` adalah untuk menghitung kemiripan antar kalimat, dan function `build_similarity_matrix` adalah untuk membuat matrix kemiripan antar kalimat.

In [43]:
def generate_summary(file_name, top_n):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    print(scores)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("\n\nSummarize Text: \n", ". ".join(summarize_text))
    print("\n")

Function di atas adalah function yang nanti digunakan untuk menghasilkan summary dari teksnya, dengan membandingkan skor dari setiap kalimat, dan memilih 3 kalimat dengan skor tertinggi untuk menjadi summarynya (sesuai dengan soal).

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
generate_summary( "data_4D.txt", 3)

﻿This paper focuses on an event for the main actors in e-tourism, called Tourism@
It is held on the French Riviera and is dedicated to new uses of Information and Communication Technology (ICT) in the tourism industry
It is a major international trade fair in Europe for innovative start up companies, high tech small and medium sized enterprises (SMEs), large multinationals and academics related to the tourism industry
Each edition of Tourism@ includes a competition aiming at awarding the best projects in terms of creativity and commitment to developing and implementing new technologies or new uses of ICT in the tourism industry
As far as Tourism@ has several unique characteristics in relation to innovation in tourism, it is an interesting case deserving an in-depth analysis
Innovation is playing an increasing role in services (Miles, 2001) and, unquestionably, is particularly important for the tourism industry (Hjalager, 2002)
Tourism has been one of the main drivers of Internet use in

Jika digunakan langsung function `generate_summary`, maka hasil summarynya langsung keluar dengan 3 kalimat. Tapi kita akan coba untuk breakdown function `generate_summary`.

In [46]:
sentences =  read_article("data_4D.txt")

﻿This paper focuses on an event for the main actors in e-tourism, called Tourism@
It is held on the French Riviera and is dedicated to new uses of Information and Communication Technology (ICT) in the tourism industry
It is a major international trade fair in Europe for innovative start up companies, high tech small and medium sized enterprises (SMEs), large multinationals and academics related to the tourism industry
Each edition of Tourism@ includes a competition aiming at awarding the best projects in terms of creativity and commitment to developing and implementing new technologies or new uses of ICT in the tourism industry
As far as Tourism@ has several unique characteristics in relation to innovation in tourism, it is an interesting case deserving an in-depth analysis
Innovation is playing an increasing role in services (Miles, 2001) and, unquestionably, is particularly important for the tourism industry (Hjalager, 2002)
Tourism has been one of the main drivers of Internet use in

Data diambil dari dataset dan langsung dibagi per kalimat sesuai function `read_article`.

In [47]:
sentence_similarity_martix = build_similarity_matrix(sentences, stop_words=stopwords.words('english'))
sentence_similarity_martix

array([[0.        , 0.        , 0.        , 0.07106691, 0.09245003,
        0.        , 0.07647191, 0.        , 0.        , 0.        ,
        0.        , 0.16666667, 0.23570226, 0.        , 0.0860663 ,
        0.        , 0.        , 0.06804138, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.11111111, 0.        , 0.08908708, 0.        ],
       [0.        , 0.        , 0.12598816, 0.30772873, 0.        ,
        0.1490712 , 0.06622662, 0.08006408, 0.13245324, 0.07216878,
        0.08703883, 0.07216878, 0.13608276, 0.09622504, 0.        ,
        0.12909944, 0.19867985, 0.11785113, 0.05661385, 0.        ,
        0.08006408, 0.09622504, 0.        , 0.        , 0.06299408,
        0.        , 0.13608276, 0.15430335, 0.09622504],
       [0.        , 0.12598816, 0.        , 0.09304842, 0.        ,
        0.11268723, 0.05006262, 0.06052275, 0.10012523, 0.10910895,
        0.13159034, 0.10910895, 0.15430335, 0.0727393 , 0.        ,
  

Dari sentence tadi, akan dibuat matrix similary antar kalimat, dimana bentuknya adalah sebuah n x n matrix dengan n adalah jumlah kalimat.

In [48]:
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)

In [49]:
scores = nx.pagerank(sentence_similarity_graph)
scores

{0: 0.02043991102987876,
 1: 0.04338972941159108,
 2: 0.035678228518778665,
 3: 0.04944682072397546,
 4: 0.03822620179920833,
 5: 0.046593143212991586,
 6: 0.024413394124403173,
 7: 0.026205497830996318,
 8: 0.03683466851248978,
 9: 0.039407135755767424,
 10: 0.034417285639524876,
 11: 0.040487234361534494,
 12: 0.06202248233822132,
 13: 0.04703075049342941,
 14: 0.028540041491896158,
 15: 0.016930687444992244,
 16: 0.04366362874916645,
 17: 0.029407970885699852,
 18: 0.029860761181353442,
 19: 0.010491633784605704,
 20: 0.023779515927556654,
 21: 0.044227016946268335,
 22: 0.017977893430345497,
 23: 0.024885561076618268,
 24: 0.04101810688059901,
 25: 0.03234622370432349,
 26: 0.033012422755995814,
 27: 0.06335109813953334,
 28: 0.015914953848255326}

In [50]:
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
print("Indexes of top 1 ranked_sentence", ranked_sentence[0])
print("Indexes of top 2 ranked_sentence", ranked_sentence[1])
print("Indexes of top 3 ranked_sentence", ranked_sentence[2])

Indexes of top 1 ranked_sentence (0.06335109813953334, ['The', 'analysis', 'of', 'the', 'Tourism@', 'trade', 'fair', 'database', 'in', 'Section', '4', 'reveals', 'the', 'important', 'innovative', 'activity', 'that', 'occurs', 'in', 'the', 'tourism', 'industry'])
Indexes of top 2 ranked_sentence (0.06202248233822132, ['The', 'aim', 'is', 'to', 'capture', 'the', 'evolution', 'of', 'innovative', 'activity', 'in', 'the', 'tourism', 'industry', 'through', 'the', 'empirical', 'analysis', 'of', 'the', 'Tourism@', 'event', 'and', 'the', 'annual', 'Tourism@', 'Awards', 'for', 'best', 'projects'])
Indexes of top 3 ranked_sentence (0.04944682072397546, ['Each', 'edition', 'of', 'Tourism@', 'includes', 'a', 'competition', 'aiming', 'at', 'awarding', 'the', 'best', 'projects', 'in', 'terms', 'of', 'creativity', 'and', 'commitment', 'to', 'developing', 'and', 'implementing', 'new', 'technologies', 'or', 'new', 'uses', 'of', 'ICT', 'in', 'the', 'tourism', 'industry'])


Kita dapat melihat skor dari tiap kalimat menggunakan function-function dari library `networkx`, dan juga akan melihat 3 kalimat dengan skor tertinggi. Ketiganya memiliki skor 0.0633, 0.062, dan 0.0494. Ketiga kalimat ini akan digabungkan menjadi 1 paragraf untuk hasil summarizationnya.

In [53]:
summarize_text = []

for i in range(3):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

print("Summarize Text: \n", ". ".join(summarize_text))

Summarize Text: 
 The analysis of the Tourism@ trade fair database in Section 4 reveals the important innovative activity that occurs in the tourism industry. The aim is to capture the evolution of innovative activity in the tourism industry through the empirical analysis of the Tourism@ event and the annual Tourism@ Awards for best projects. Each edition of Tourism@ includes a competition aiming at awarding the best projects in terms of creativity and commitment to developing and implementing new technologies or new uses of ICT in the tourism industry


Hasil teks summarization dengan 3 kalimat adalah:<br><br>
 The analysis of the Tourism@ trade fair database in Section 4 reveals the important innovative activity that occurs in the tourism industry. The aim is to capture the evolution of innovative activity in the tourism industry through the empirical analysis of the Tourism@ event and the annual Tourism@ Awards for best projects. Each edition of Tourism@ includes a competition aiming at awarding the best projects in terms of creativity and commitment to developing and implementing new technologies or new uses of ICT in the tourism industry
